# Practical Exam: Grocery Store Sales

FoodYum is a grocery store chain that is based in the United States.

Food Yum sells items such as produce, meat, dairy, baked goods, snacks, and other household food staples.

As food costs rise, FoodYum wants to make sure it keeps stocking products in all categories that cover a range of prices to ensure they have stock for a broad range of customers. 

## Data

The data is available in the table [`products`.]()

The dataset contains records of customers for their last full year of the loyalty program.

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with 2022. |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

# Task 1

Last year (2022) there was a bug in the product system. For some products that were added in that year, the `year_added` value was not set in the data. As the year the product was added may have an impact on the price of the product, this is important information to have. 

Write a query to determine how many products have the `year_added` value missing. Your output should be a single column, `missing_year`, with a single row giving the number of missing values.

In [1]:
select count(*) as missing_year
from public.products
where year_added is null;


,missing_year
0,170


# Task 2

Given what you know about the year added data, you need to make sure all of the data is clean before you start your analysis. The table below shows what the data should look like. 

Write a query to ensure the product data matches the description provided. Do not update the original table.  

| Column Name | Criteria                                                |
|-------------|---------------------------------------------------------|
|product_id | Nominal. The unique identifier of the product. </br>Missing values are not possible due to the database structure.|
| product_type | Nominal. The product category type of the product, one of 5 values (Produce, Meat, Dairy, Bakery, Snacks). </br>Missing values should be replaced with “Unknown”. |
| brand | Nominal. The brand of the product. One of 7 possible values. </br>Missing values should be replaced with “Unknown”. |
| weight | Continuous. The weight of the product in grams. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median weight. |
| price | Continuous. The price the product is sold at, in US dollars. This can be any positive value, rounded to 2 decimal places. </br>Missing values should be replaced with the overall median price. |
| average_units_sold | Discrete. The average number of units sold each month. This can be any positive integer value. </br>Missing values should be replaced with 0. |
| year_added | Nominal. The year the product was first added to FoodYum stock.</br>Missing values should be replaced with last year (2022). |
| stock_location | Nominal. The location that stock originates. This can be one of four warehouse locations, A, B, C or D </br>Missing values should be replaced with “Unknown”. |

In [2]:
select data_type,column_name
from information_schema.columns
where table_name = 'products';


create temp table clean_data
(product_id int not null,
product_type text,
brand text,
weight text,
price numeric,
average_units_sold int,
year_added int,
stock_location text);

insert into clean_data
select * from public.products;

select distinct product_type from clean_data;
select distinct brand from clean_data;--'-'
select max(weight),min(weight) from clean_data;
select max(price) , min (price) from clean_data;
select max(average_units_sold) , min (average_units_sold) from clean_data;
select average_units_sold from clean_data where average_units_sold is null;
select year_added from clean_data where year_added is null;--null
select distinct stock_location from clean_data;

update clean_data
set product_type = 'Unknown'
where product_type is null;

update clean_data
set brand = 'Unknown'
where brand = '-';

update clean_data
set average_units_sold = 0
where average_units_sold is null;

update clean_data
set year_added = 2022
where year_added is null;

update clean_data
set stock_location = 'Unknown'
where stock_location is null;

select 
	product_id,
	product_type,
	brand,
	Case
        when weight is null then round(avg(split_part(weight,' ',1):: numeric) ,2)
        else round(split_part(weight,' ',1):: numeric ,2)
        end as weight,
	ROUND(COALESCE(price :: numeric, avg(price:: numeric)),2) as price,
	average_units_sold,
	year_added,
	Upper(stock_location) as stock_location
from clean_data
group by  PRODUCT_ID,PRODUCT_TYPE,BRAND,WEIGHT,PRICE,AVERAGE_UNITS_SOLD,YEAR_ADDED,STOCK_LOCATION;



,product_id,product_type,brand,weight,price,average_units_sold,year_added,stock_location
0,1160,Dairy,GoldTree,442.82,13.13,22,2017,C
1,813,Meat,GoldTree,488.12,16.12,25,2015,C
2,50,Dairy,GoldTree,548.67,12.75,22,2022,C
3,758,Dairy,GoldTree,485.49,13.02,22,2020,A
4,1240,Produce,GoldTree,606.01,8.11,21,2019,C
...,...,...,...,...,...,...,...,...
1695,1362,Meat,SmoothTaste,458.21,14.33,26,2021,B
1696,138,Meat,StandardYums,541.56,11.79,31,2018,A
1697,1610,Bakery,TastyTreat,450.17,10.03,16,2019,C
1698,1467,Produce,GoldTree,555.46,7.92,21,2015,C


# Task 3

To find out how the range varies for each product type, your manager has asked you to determine the minimum and maximum values for each product type.   

Write a query to return the `product_type`, `min_price` and `max_price` columns. 

In [3]:
with min_max_product as 
( 
select product_type,min(price) as min_price, max(price) as max_price
from public.products
group by product_type
	)
	
select * from min_max_product;

,product_type,min_price,max_price
0,Snacks,5.20,10.72
1,Produce,3.46,8.78
2,Dairy,8.33,13.97
3,Bakery,6.26,11.88
4,Meat,11.48,16.98


# Task 4

The team want to look in more detail at meat and dairy products where the average units sold was greater than ten. 

Write a query to return the `product_id`, `price` and `average_units_sold` of the rows of interest to the team. 

In [4]:
with average_price_product as
(select product_id,price, average_units_sold
from public.products
where (product_type = 'Dairy' or product_type = 'Meat')
 group by 1,2,3
 having average_units_sold > 10)
 
 select * from average_price_product ;


,product_id,price,average_units_sold
0,1143,10.86,22
1,935,15.87,26
2,545,13.14,30
3,332,11.13,22
4,115,11.06,23
...,...,...,...
693,125,12.84,23
694,1335,12.93,24
695,1619,14.04,28
696,1535,16.11,25
